In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

OUTDIR = Path("./prepared")
OUTDIR.mkdir(parents=True, exist_ok=True)

In [18]:
df_yields = pd.read_csv("./data/Yields.csv")
df_yields.columns = df_yields.columns.str.strip()
print(df_yields.shape)

# Basic sanity checks
n_crops = df_yields["Crop"].nunique(dropna=True)
n_munis = df_yields["Municipality"].nunique(dropna=True)
print(f"- number of different crops: {n_crops}")
print(f"- number of different municipalities: {n_munis}")

df_yields.head()


(44448, 9)
- number of different crops: 53
- number of different municipalities: 99


,Year,Municipality,Crop,Variety,Farms,Acres,Yield/Acre,t,i
0,1996,ALEXANDER,ARGENTINE CANOLA,INNOVATOR (HCN 92) (LT),3,575.0,0.579,1,1
1,1996,ALEXANDER,ARGENTINE CANOLA,QUANTUM (91-21864 NA),6,1125.0,0.780,1,1
2,1996,ALEXANDER,ARGENTINE CANOLA,45A71 (NS1471)(ST),4,1012.0,0.756,1,1
3,1996,ALEXANDER,BARLEY,ROBUST,9,1247.0,1.607,1,1
4,1996,ALEXANDER,CANARYSEED,KEET,3,535.0,0.184,1,1


In [19]:
df_weather = pd.read_csv("./data/Weather Reanalysis.csv")
df_weather.columns = df_weather.columns.str.strip()
print(df_weather.shape)

n_munis = df_weather["Municipality"].nunique(dropna=True)
print(f"- number of different municipalities: {n_munis}")
df_weather.head()


(490896, 9)
- number of different municipalities: 84


,Municipality,Date/Time,Year,Month,Day,Max_Temp,Min_Temp,Mean_Temp,Ptol
0,ALEXANDER,1/1/96,1996,1,1,-10.416400,-21.008989,-15.712695,-1.370000e-10
1,ALEXANDER,1/2/96,1996,1,2,-15.238113,-18.910885,-17.074499,-1.370000e-10
2,ALEXANDER,1/3/96,1996,1,3,-13.913316,-21.768749,-17.841032,1.088851e-03
3,ALEXANDER,1/4/96,1996,1,4,-22.517648,-28.702039,-25.609844,-1.370000e-10
4,ALEXANDER,1/5/96,1996,1,5,-23.918762,-30.488448,-27.203605,-1.370000e-10


In [20]:
YEARS = range(1996, 2012)  # 1996–2011

def prepare_yields_year_muni_crop(df):
    """Aggregate Yields.csv to one row per (Year, Municipality, Crop).

    Mean_Yield is the mean yield across varieties for the same (Year, Municipality, Crop).
    Recommended definition: area-weighted mean of Yield/Acre using Acres:
        Mean_Yield = sum(Yield/Acre * Acres) / sum(Acres)
    If Acres is missing/zero for a group, fall back to simple mean of Yield/Acre.
    """
    y = df.copy()
    y.columns = y.columns.str.strip()

    # Required columns check
    required = {"Year", "Municipality", "Crop", "Yield/Acre"}
    missing = required - set(y.columns)
    if missing:
        raise ValueError(f"Missing required columns in Yields.csv: {sorted(missing)}")

    # Types
    y["Year"] = pd.to_numeric(y["Year"], errors="coerce")
    y["Yield/Acre"] = pd.to_numeric(y["Yield/Acre"], errors="coerce")

    if "Acres" in y.columns:
        y["Acres"] = pd.to_numeric(y["Acres"], errors="coerce")
    else:
        y["Acres"] = np.nan  # will trigger fallback to simple mean

    # Clean strings
    y["Municipality"] = y["Municipality"].astype(str).str.strip().str.upper()
    y["Crop"] = y["Crop"].astype(str).str.strip()

    # Filter study window and remove invalid rows
    y = y[y["Year"].between(min(YEARS), max(YEARS))].copy()
    y = y.dropna(subset=["Year", "Municipality", "Crop", "Yield/Acre"])

    # Weighted mean yield per acre (weights = Acres)
    y["_acres"] = y["Acres"].fillna(0)
    y["_prod"] = y["Yield/Acre"] * y["_acres"]

    grouped = (
        y.groupby(["Year", "Municipality", "Crop"], as_index=False)
         .agg(
             acres_sum=("_acres", "sum"),
             prod_sum=("_prod", "sum"),
             yield_mean_simple=("Yield/Acre", "mean"),
         )
    )

    grouped["Mean_Yield"] = np.where(
        grouped["acres_sum"] > 0,
        grouped["prod_sum"] / grouped["acres_sum"],
        grouped["yield_mean_simple"]
    )

    out = grouped[["Year", "Municipality", "Crop", "Mean_Yield"]].copy()
    out["Year"] = out["Year"].astype(int)
    return out


In [21]:
yields_by_crop = prepare_yields_year_muni_crop(df_yields)

print("Prepared yields_by_crop:", yields_by_crop.shape)
print("Unique key check (Year, Municipality, Crop):", yields_by_crop.duplicated(["Year","Municipality","Crop"]).sum())

yields_by_crop.to_csv(OUTDIR / "yields_by_crop.csv", index=False)
yields_by_crop.head()


Prepared yields_by_crop: (14476, 4)
Unique key check (Year, Municipality, Crop): 0


,Year,Municipality,Crop,Mean_Yield
0,1996,ALEXANDER,ARGENTINE CANOLA,0.728428
1,1996,ALEXANDER,BARLEY,1.607000
2,1996,ALEXANDER,CANARYSEED,0.184000
3,1996,ALEXANDER,OATS,1.215000
4,1996,ALEXANDER,POLISH CANOLA,0.278000


In [22]:
GROW_START, GROW_END = ("05-01", "10-31")   # May–Oct

def parse_date_us_2digit_year(s):
    # Robustly parse strings like "1/1/96" -> 1996-01-01
    return pd.to_datetime(s, format="%m/%d/%y", errors="coerce")

def load_weather(e):
    e = e.copy()
    e.columns = e.columns.str.strip()
    e["Municipality"] = e["Municipality"].astype(str).str.strip().str.upper()

    # Parse date
    e["__date"] = parse_date_us_2digit_year(e["Date/Time"])

    # Filter to study window
    e = e[e["Year"].between(min(YEARS), max(YEARS))].copy()

    # Growing season mask (May–Oct by month/day)
    e["__mmdd"] = e["__date"].dt.strftime("%m-%d")
    season = (e["__mmdd"] >= GROW_START) & (e["__mmdd"] <= GROW_END)
    e = e.loc[season].copy()

    return e

def weather_indices(e,
                thresholds=dict(hot=30.0, frost=0.0, wet=0.001, gdd_base=5.0, gdd_cap=30.0)):
    """
    Build annual (Municipality, Year) indices for May–Oct:
      - Averages: mean Tmin/Tmax/Tmean
      - Extremes: min Tmin, max Tmax, max daily precip
      - Totals/Counts: precip total, #hot days (Tmax>=hot), #frost days (Tmin<=frost), #wet days (P>=wet)
      - Degree-days: capped GDD using daily mean
    """
    e = load_weather(e)

    # Ensure numeric
    for col in ["Min_Temp", "Max_Temp", "Mean_Temp", "Ptol"]:
        if col in e.columns:
            e[col] = pd.to_numeric(e[col], errors="coerce")

    # Degree-days (cap at gdd_cap and floor at base)
    tmean = e["Mean_Temp"]
    base = thresholds["gdd_base"]; cap = thresholds["gdd_cap"]
    gdd = np.clip(tmean - base, 0, cap - base)
    e["GDD"] = gdd

    # Extreme-day counts
    e["HotDays"]   = (e["Max_Temp"] >= thresholds["hot"]).astype(int)
    e["FrostDays"] = (e["Min_Temp"] <= thresholds["frost"]).astype(int)
    e["WetDays"]   = (e["Ptol"] >= thresholds["wet"]).astype(int)

    # Aggregations
    grp = e.groupby(["Municipality", "Year"])
    idx = grp.agg({
        "Min_Temp": ["mean", "min"],
        "Max_Temp": ["mean", "max"],
        "Mean_Temp":["mean"],
        "Ptol": ["sum", "max"],
        "GDD":    ["sum"],
        "HotDays":["sum"],
        "FrostDays":["sum"],
        "WetDays":["sum"],
    }).reset_index()

    # Flatten columns
    idx.columns = ["Municipality","Year",
                   "Tmin_mean","Tmin_min",
                   "Tmax_mean","Tmax_max",
                   "Tmean_mean",
                   "Ptot_sum","Ptot_max",
                   "GDD_sum",
                   "HotDays_sum","FrostDays_sum","WetDays_sum"]
    return idx

import numpy as np
import pandas as pd


def weather_indices_biweekly(
    weather_df: pd.DataFrame,
    *,
    municipality_col: str = "Municipality",
    year_col: str = "Year",
    month_col: str = "Month",
    day_col: str = "Day",
    max_temp_col: str = "Max_Temp",
    min_temp_col: str = "Min_Temp",
    mean_temp_col: str = "Mean_Temp",
    precip_total_col: str | None = "Ptol",          # total precip (rain+snow), daily
    rain_col: str | None = None,                   # if you have it (daily)
    snow_col: str | None = None,                   # if you have it (daily)
    base_temp_c: float = 18.0,                     # base for HDD/CDD (looks like 18°C in your biweekly file)
    clip_precip_at_zero: bool = True,              # your Ptol has tiny negative epsilons
) -> pd.DataFrame:
    """
    Build biweekly (month × Q1/Q2) climate columns like:
      '01_Q1_Max_Temp', '01_Q1_Mean_Temp', '01_Q1_Min_Temp',
      '01_Q1_Total Rain (mm)', '01_Q1_Total Snow', '01_Q1_Ptol',
      '01_Q1_HDD', '01_Q1_CDD', ... for all months.

    Output grain: (Year, Municipality) wide table.
    """

    df = weather_df.copy()

    # ---- basic checks ----
    required = [municipality_col, year_col, month_col, day_col, mean_temp_col]
    for c in required:
        if c not in df.columns:
            raise ValueError(f"Missing required column: {c}")

    # ensure numeric
    for c in [year_col, month_col, day_col, max_temp_col, min_temp_col, mean_temp_col]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    # ---- define half-month bucket ----
    df["Half"] = np.where(df[day_col] <= 15, "Q1", "Q2")
    df["Month2"] = df[month_col].astype("Int64")  # keep NA-safe int

    # ---- HDD / CDD on daily mean temp ----
    tmean = df[mean_temp_col]
    df["_HDD"] = np.maximum(0.0, base_temp_c - tmean)
    df["_CDD"] = np.maximum(0.0, tmean - base_temp_c)

    # ---- precip handling ----
    # We try to reproduce your biweekly file columns:
    #   Total Rain (mm), Total Snow, Ptol
    # If only Ptol exists (like your Weather Reanalysis.csv), we set:
    #   Total Rain (mm) = Ptol, Total Snow = 0
    if precip_total_col is not None and precip_total_col in df.columns:
        df[precip_total_col] = pd.to_numeric(df[precip_total_col], errors="coerce")
        if clip_precip_at_zero:
            df[precip_total_col] = df[precip_total_col].clip(lower=0)

    if rain_col is not None and rain_col in df.columns:
        df[rain_col] = pd.to_numeric(df[rain_col], errors="coerce")
        if clip_precip_at_zero:
            df[rain_col] = df[rain_col].clip(lower=0)

    if snow_col is not None and snow_col in df.columns:
        df[snow_col] = pd.to_numeric(df[snow_col], errors="coerce")
        if clip_precip_at_zero:
            df[snow_col] = df[snow_col].clip(lower=0)

    if rain_col is None or rain_col not in df.columns:
        if precip_total_col is not None and precip_total_col in df.columns:
            df["_TotalRain"] = df[precip_total_col]
        else:
            df["_TotalRain"] = np.nan
    else:
        df["_TotalRain"] = df[rain_col]

    if snow_col is None or snow_col not in df.columns:
        df["_TotalSnow"] = 0.0
    else:
        df["_TotalSnow"] = df[snow_col]

    if precip_total_col is None or precip_total_col not in df.columns:
        # if no Ptol given but rain/snow are present
        df["_Ptol"] = df["_TotalRain"] + df["_TotalSnow"]
    else:
        df["_Ptol"] = df[precip_total_col]

    # ---- aggregation per (Year, Municipality, Month, Half) ----
    # Convention (consistent with your file + typical climate summaries):
    #   *_Max_Temp = mean of daily Max_Temp over the period
    #   *_Min_Temp = mean of daily Min_Temp
    #   *_Mean_Temp = mean of daily Mean_Temp
    #   totals (rain/snow/Ptol) = sum over days
    grp_cols = [year_col, municipality_col, "Month2", "Half"]
    agg_dict = {
        "_HDD": "sum",
        "_CDD": "sum",
        "_TotalRain": "sum",
        "_TotalSnow": "sum",
        "_Ptol": "sum",
    }
    if max_temp_col in df.columns:
        agg_dict[max_temp_col] = "mean"
    if min_temp_col in df.columns:
        agg_dict[min_temp_col] = "mean"
    if mean_temp_col in df.columns:
        agg_dict[mean_temp_col] = "mean"

    bi = df.groupby(grp_cols, dropna=False).agg(agg_dict).reset_index()

    # rename metrics to match your biweekly column naming
    rename_map = {}
    if max_temp_col in bi.columns:
        rename_map[max_temp_col] = "Max_Temp"
    if min_temp_col in bi.columns:
        rename_map[min_temp_col] = "Min_Temp"
    if mean_temp_col in bi.columns:
        rename_map[mean_temp_col] = "Mean_Temp"

    rename_map.update({
        "_TotalRain": "Total Rain (mm)",
        "_TotalSnow": "Total Snow",
        "_Ptol": "Ptol",
        "_HDD": "HDD",
        "_CDD": "CDD",
    })
    bi = bi.rename(columns=rename_map)

    # ---- wide pivot: one row per (Year, Municipality) ----
    bi["MonthHalf"] = bi["Month2"].apply(lambda m: f"{int(m):02d}" if pd.notna(m) else "NA") + "_" + bi["Half"]
    id_cols = [year_col, municipality_col]

    value_cols = [c for c in bi.columns if c not in (id_cols + ["Month2", "Half", "MonthHalf"])]

    wide = (
        bi.pivot_table(index=id_cols, columns="MonthHalf", values=value_cols, aggfunc="first")
          .sort_index(axis=1, level=1)
    )

    # flatten MultiIndex columns -> "MM_Qx_<Metric>"
    wide.columns = [f"{mh}_{metric}" for metric, mh in wide.columns]
    wide = wide.reset_index()

    return wide



In [23]:
weather_idx_raw = weather_indices_biweekly(df_weather)
weather_idx_raw.to_csv(OUTDIR / "weather_indices_raw.csv", index=False)
print("Prepared weather indices:", weather_idx_raw.shape)
weather_idx_raw.head()


Prepared weather indices: (1344, 194)


,Year,Municipality,01_Q1_CDD,01_Q1_HDD,01_Q1_Max_Temp,01_Q1_Mean_Temp,01_Q1_Min_Temp,01_Q1_Ptol,01_Q1_Total Rain (mm),01_Q1_Total Snow,...,12_Q1_Total Rain (mm),12_Q1_Total Snow,12_Q2_CDD,12_Q2_HDD,12_Q2_Max_Temp,12_Q2_Mean_Temp,12_Q2_Min_Temp,12_Q2_Ptol,12_Q2_Total Rain (mm),12_Q2_Total Snow
0,1996,ALEXANDER,0.0,497.027223,-12.156704,-15.135148,-18.113592,0.007748,0.007748,0.0,...,0.027243,0.0,0.0,663.597773,-20.892594,-23.474861,-26.057127,0.014210,0.014210,0.0
1,1996,ALONSA,0.0,478.850815,-10.803989,-13.923388,-17.042786,0.011911,0.011911,0.0,...,0.016316,0.0,0.0,651.589939,-19.893860,-22.724371,-25.554883,0.015288,0.015288,0.0
2,1996,ARGYLE,0.0,479.586678,-10.933534,-13.972445,-17.011357,0.010000,0.010000,0.0,...,0.010000,0.0,0.0,652.117702,-19.725936,-22.757356,-25.788776,0.020000,0.020000,0.0
3,1996,ARMSTRONG,0.0,496.165580,-12.226368,-15.077705,-17.929042,0.000000,0.000000,0.0,...,0.010000,0.0,0.0,667.698280,-20.885745,-23.731142,-26.576540,0.000000,0.000000,0.0
4,1996,BIFROST-RIVERTON,0.0,496.165580,-12.226368,-15.077705,-17.929042,0.009514,0.009514,0.0,...,0.023289,0.0,0.0,667.698280,-20.885745,-23.731142,-26.576540,0.009415,0.009415,0.0


In [24]:
yields_weather_by_crop = yields_by_crop.merge(
    weather_idx_raw,
    on=["Municipality", "Year"],
    how="inner"
)

print("Merged yields + weather:", yields_weather_by_crop.shape)

# Report missing climate coverage if using left join (optional diagnostics)
# missing = yields_by_crop.merge(weather_idx_raw, on=["Municipality","Year"], how="left", indicator=True)
# print("Rows without climate:", (missing["_merge"] == "left_only").sum())

yields_weather_by_crop.to_csv(OUTDIR / "yields_weather_by_crop.csv", index=False)
yields_weather_by_crop.head()


Merged yields + weather: (11721, 196)


,Year,Municipality,Crop,Mean_Yield,01_Q1_CDD,01_Q1_HDD,01_Q1_Max_Temp,01_Q1_Mean_Temp,01_Q1_Min_Temp,01_Q1_Ptol,...,12_Q1_Total Rain (mm),12_Q1_Total Snow,12_Q2_CDD,12_Q2_HDD,12_Q2_Max_Temp,12_Q2_Mean_Temp,12_Q2_Min_Temp,12_Q2_Ptol,12_Q2_Total Rain (mm),12_Q2_Total Snow
0,1996,ALEXANDER,ARGENTINE CANOLA,0.728428,0.0,497.027223,-12.156704,-15.135148,-18.113592,0.007748,...,0.027243,0.0,0.0,663.597773,-20.892594,-23.474861,-26.057127,0.01421,0.01421,0.0
1,1996,ALEXANDER,BARLEY,1.607000,0.0,497.027223,-12.156704,-15.135148,-18.113592,0.007748,...,0.027243,0.0,0.0,663.597773,-20.892594,-23.474861,-26.057127,0.01421,0.01421,0.0
2,1996,ALEXANDER,CANARYSEED,0.184000,0.0,497.027223,-12.156704,-15.135148,-18.113592,0.007748,...,0.027243,0.0,0.0,663.597773,-20.892594,-23.474861,-26.057127,0.01421,0.01421,0.0
3,1996,ALEXANDER,OATS,1.215000,0.0,497.027223,-12.156704,-15.135148,-18.113592,0.007748,...,0.027243,0.0,0.0,663.597773,-20.892594,-23.474861,-26.057127,0.01421,0.01421,0.0
4,1996,ALEXANDER,POLISH CANOLA,0.278000,0.0,497.027223,-12.156704,-15.135148,-18.113592,0.007748,...,0.027243,0.0,0.0,663.597773,-20.892594,-23.474861,-26.057127,0.01421,0.01421,0.0
